In [19]:
import pandas as pd
import numpy as np

data = pd.read_csv("polygonDataExceptVertices.csv")
vertices = pd.read_csv("polygonVertices_PixelCoordinates.csv")


print(data.shape)
print(vertices.shape)

(19435, 17)
(19433, 294)


In [20]:
data.head()

,polygon_id,centroid_latitude,centroid_longitude,centroid_latitude_pixels,centroid_longitude_pixels,city,area_pixels,area_meters,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude,datum,projection_zone,resolution,jaccard_index
0,1,36.926310,-119.840555,107.618458,3286.151487,Fresno,136.192872,1513.254134,11ska460890,36.926336,-119.851622,36.913233,-119.8343,NAD83,11,0.3,0.914020
1,2,36.926477,-119.840561,45.977659,3286.352946,Fresno,155.511714,1727.907934,11ska460890,36.926336,-119.851622,36.913233,-119.8343,NAD83,11,0.3,0.829071
2,3,36.926542,-119.840506,22.280851,3303.465657,Fresno,111.796591,1242.184349,11ska460890,36.926336,-119.851622,36.913233,-119.8343,NAD83,11,0.3,0.937961
3,4,36.921008,-119.842847,2048.362567,2547.366116,Fresno,62.004008,688.933420,11ska460890,36.926336,-119.851622,36.913233,-119.8343,NAD83,11,0.3,0.842634
4,5,36.920976,-119.842906,2060.014890,2529.504997,Fresno,95.480150,1060.890554,11ska460890,36.926336,-119.851622,36.913233,-119.8343,NAD83,11,0.3,0.890998


In [21]:
data.tail()

,polygon_id,centroid_latitude,centroid_longitude,centroid_latitude_pixels,centroid_longitude_pixels,city,area_pixels,area_meters,image_name,nw_corner_of_image_latitude,nw_corner_of_image_longitude,se_corner_of_image_latitude,se_corner_of_image_longitude,datum,projection_zone,resolution,jaccard_index
19430,19859,38.004757,-121.367798,4667.135295,4327.942553,Stockton,7.922866,88.031847,10sfh420075,38.017578,-121.382306,38.003825,-121.365522,NAD83,10,0.3,0.768137
19431,19860,38.004803,-121.367693,4649.417351,4358.422379,Stockton,16.183635,179.818164,10sfh420075,38.017578,-121.382306,38.003825,-121.365522,NAD83,10,0.3,0.855938
19432,19861,38.004746,-121.367725,4670.683884,4349.368715,Stockton,16.618386,184.648735,10sfh420075,38.017578,-121.382306,38.003825,-121.365522,NAD83,10,0.3,0.871239
19433,19862,38.013145,-121.367647,1563.736866,4317.773765,Stockton,6.051385,67.237616,10sfh420075,38.017578,-121.382306,38.003825,-121.365522,NAD83,10,0.3,0.000000
19434,19863,38.011124,-121.368179,2314.112357,4175.227893,Stockton,43.992519,488.805771,10sfh420075,38.017578,-121.382306,38.003825,-121.365522,NAD83,10,0.3,0.000000


In [22]:
vertices.head()

,polygon_id,number_vertices,lon1,lat1,lon2,lat2,lon3,lat3,lon4,lat4,...,lon142,lat142,lon143,lat143,lon144,lat144,lon145,lat145,lon146,lat146
0,1,8,3360.495069,131.631164,3249.197710,87.985141,3213.750000,73.850000,3210.450000,81.950000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,2,9,3361.153846,69.615385,3217.615385,12.846154,3217.440417,13.096107,3215.850000,12.450000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,3,11,3358.015726,48.136863,3358.050000,48.150000,3360.742998,42.379290,3362.122288,39.467456,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,4,9,2571.591716,2068.049310,2571.021266,2067.393899,2571.350000,2067.150000,2531.150000,2021.550000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,5,9,2563.781065,2091.398422,2504.416174,2021.674556,2503.979640,2022.048728,2503.550000,2021.550000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
#vertices.tail()
#vertices['number_vertices'].max()
vertices['polygon_id'][vertices['number_vertices']==146]

1251    1287
Name: polygon_id, dtype: int64

In [44]:
clean = pd.melt(vertices, id_vars = ['polygon_id','number_vertices']) #melting all the vertice co-ordinates
clean = clean.dropna() #drop the rows that don't have a value
clean['value'] = clean['value'].map(lambda x: int(round(x)))
clean['pixel_id'] = clean['variable'].map(lambda x: int(x.replace('lon','').replace('lat','')))
clean.drop('variable', axis=1, inplace=True)
'''    
    # Assign each row with the pair it belongs to 
pairs = zip(range(1,292,2), range(2,293,2))
pairs = [pair for double in zip(pairs, pairs) for pair in double]    
clean['pair'] = clean['pixel_id'].map(lambda x: pairs[x - 1])
    
    # Seperate odd/even and merge together
even = clean[(clean['pixel_id'] % 2 == 0)]
odd = clean[~(clean['pixel_id'] % 2 == 0)]
    
pix_merge = odd.merge(even, on=['polygon_id','number_vertices','pair'])
pix_merge['coordinate'] = map(lambda x, y: (x, y), pix_merge['value_x'].values, pix_merge['value_y'].values)
    
    # Recombine all coordinate pairs by polygon id 
polygon_coords = pix_merge[['polygon_id','coordinate']]
polygon_coords = pd.pivot_table(polygon_coords, index='polygon_id', values='coordinate',
                                aggfunc=lambda x: x.values.tolist())
    
polygon_coords = polygon_coords.reset_index()
polygon_coords.columns = ['polygon_id','polygon_vertices']
'''
clean.head()

,polygon_id,number_vertices,value,pixel_id
0,1,8,3360,1
1,2,9,3361,1
2,3,11,3358,1
3,4,9,2572,1
4,5,9,2564,1


12.141511861266917